In [4]:
!git clone https://github.com/KumarAnurag31/real-estate-business-intelligence.git
%cd real-estate-business-intelligence

fatal: destination path 'real-estate-business-intelligence' already exists and is not an empty directory.
/content/real-estate-business-intelligence


In [6]:
!pip install streamlit pandas plotly pyodbc

import streamlit as st
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime
import numpy as np

# Only import pyodbc if available (optional for CSV-only usage)
try:
    import pyodbc
    PYODBC_AVAILABLE = True
except ImportError:
    PYODBC_AVAILABLE = False
    st.warning("⚠️ pyodbc not installed. SQL Server connection will not be available.")

# Page configuration
st.set_page_config(
    page_title="Real Estate USA Dashboard",
    page_icon="🏠",
    layout="wide",
    initial_sidebar_state="expanded"
)

# Enhanced Custom CSS with modern design
st.markdown("""
    <style>
    .main-header {
        font-size: 3.5rem;
        font-weight: bold;
        background: linear-gradient(120deg, #1f77b4, #ff7f0e);
        -webkit-background-clip: text;
        -webkit-text-fill-color: transparent;
        text-align: center;
        margin-bottom: 2rem;
        padding: 1rem;
    }

    .stMetric {
        background-color: #f8f9fa;
        padding: 1rem;
        border-radius: 10px;
        box-shadow: 0 4px 6px rgba(0,0,0,0.1);
    }

    .info-box {
        background-color: #e7f3ff;
        border-left: 5px solid #1f77b4;
        padding: 1rem;
        margin: 1rem 0;
        border-radius: 5px;
    }
    </style>
""", unsafe_allow_html=True)

# Database connection function
@st.cache_resource
def get_database_connection():
    """Create database connection to SQL Server"""
    if not PYODBC_AVAILABLE:
        return None

    try:
        if hasattr(st, 'secrets') and 'server' in st.secrets:
            connection_string = (
                f"DRIVER={{ODBC Driver 17 for SQL Server}};"
                f"SERVER={st.secrets['server']};"
                f"DATABASE={st.secrets['database']};"
                f"UID={st.secrets['username']};"
                f"PWD={st.secrets['password']};"
                f"Encrypt=yes;"
                f"TrustServerCertificate=no;"
                f"Connection Timeout=30;"
            )
            conn = pyodbc.connect(connection_string)
            return conn
        else:
            return None
    except Exception as e:
        st.error(f"❌ Database connection error: {str(e)}")
        return None

# Load data from SQL Server
@st.cache_data(ttl=3600)
def load_data_from_db(_conn, custom_query=None):
    """Load data from SQL Server database"""
    try:
        if custom_query:
            query = custom_query
        else:
            query = """
            SELECT
                PropertyType,
                State,
                City,
                SaleDate,
                SalePrice,
                Bedrooms,
                Bathrooms,
                SquareFeet,
                YearBuilt
            FROM RealEstateSales
            WHERE SaleDate >= DATEADD(year, -5, GETDATE())
            ORDER BY SaleDate DESC
            """

        df = pd.read_sql(query, _conn)
        df['SaleDate'] = pd.to_datetime(df['SaleDate'])
        df['Year'] = df['SaleDate'].dt.year
        df['Month'] = df['SaleDate'].dt.month
        df['MonthName'] = df['SaleDate'].dt.strftime('%B')
        df['Quarter'] = df['SaleDate'].dt.quarter
        df['PricePerSqFt'] = (df['SalePrice'] / df['SquareFeet']).round(2)
        df['PropertyAge'] = datetime.now().year - df['YearBuilt']

        return df
    except Exception as e:
        st.error(f"❌ Error loading data: {str(e)}")
        return None

# Load data from CSV
@st.cache_data
def load_data_from_csv():
    """Load data from CSV files"""
    try:
        df = pd.read_csv('Quered data property sold.csv')

        if 'SaleDate' in df.columns:
            df['SaleDate'] = pd.to_datetime(df['SaleDate'], errors='coerce')
            df['Year'] = df['SaleDate'].dt.year
            df['Month'] = df['SaleDate'].dt.month
            df['MonthName'] = df['SaleDate'].dt.strftime('%B')

        if 'SalePrice' in df.columns and 'SquareFeet' in df.columns:
            df['PricePerSqFt'] = (df['SalePrice'] / df['SquareFeet']).round(2)

        return df
    except FileNotFoundError:
        st.warning("⚠️ CSV file 'Quered data property sold.csv' not found.")
        return None
    except Exception as e:
        st.error(f"❌ Error loading CSV: {str(e)}")
        return None

# Generate sample data
@st.cache_data
def generate_sample_data():
    """Generate realistic sample real estate data"""
    np.random.seed(42)

    states = ['California', 'Texas', 'Florida', 'New York', 'Illinois',
              'Pennsylvania', 'Ohio', 'Georgia', 'North Carolina', 'Michigan']
    cities = {
        'California': ['Los Angeles', 'San Francisco', 'San Diego', 'Sacramento'],
        'Texas': ['Houston', 'Dallas', 'Austin', 'San Antonio'],
        'Florida': ['Miami', 'Tampa', 'Orlando', 'Jacksonville'],
        'New York': ['New York City', 'Buffalo', 'Rochester', 'Albany'],
        'Illinois': ['Chicago', 'Aurora', 'Naperville', 'Rockford']
    }
    property_types = ['Single Family', 'Condo', 'Townhouse', 'Multi-Family', 'Luxury Estate']

    n_records = 1500
    data = []

    for _ in range(n_records):
        state = np.random.choice(states)
        city = np.random.choice(cities.get(state, ['Unknown City']))
        prop_type = np.random.choice(property_types, p=[0.4, 0.25, 0.2, 0.1, 0.05])

        if prop_type == 'Luxury Estate':
            bedrooms = np.random.randint(4, 8)
            bathrooms = np.random.randint(3, 7)
            sqft = np.random.randint(3000, 8000)
            base_price = np.random.randint(800000, 3000000)
        elif prop_type == 'Multi-Family':
            bedrooms = np.random.randint(3, 6)
            bathrooms = np.random.randint(2, 5)
            sqft = np.random.randint(2000, 4500)
            base_price = np.random.randint(400000, 1200000)
        elif prop_type == 'Condo':
            bedrooms = np.random.randint(1, 4)
            bathrooms = np.random.randint(1, 3)
            sqft = np.random.randint(600, 2000)
            base_price = np.random.randint(150000, 600000)
        elif prop_type == 'Townhouse':
            bedrooms = np.random.randint(2, 5)
            bathrooms = np.random.randint(2, 4)
            sqft = np.random.randint(1200, 2800)
            base_price = np.random.randint(250000, 700000)
        else:
            bedrooms = np.random.randint(2, 6)
            bathrooms = np.random.randint(1, 4)
            sqft = np.random.randint(1000, 3500)
            base_price = np.random.randint(200000, 900000)

        year_built = np.random.randint(1960, 2024)

        data.append({
            'State': state,
            'City': city,
            'PropertyType': prop_type,
            'SalePrice': base_price,
            'Bedrooms': bedrooms,
            'Bathrooms': bathrooms,
            'SquareFeet': sqft,
            'YearBuilt': year_built
        })

    df = pd.DataFrame(data)
    dates = pd.date_range(end=datetime.now(), periods=n_records, freq='D')
    df['SaleDate'] = dates
    df['SaleDate'] = pd.to_datetime(df['SaleDate'])
    df['Year'] = df['SaleDate'].dt.year
    df['Month'] = df['SaleDate'].dt.month
    df['MonthName'] = df['SaleDate'].dt.strftime('%B')
    df['Quarter'] = df['SaleDate'].dt.quarter
    df['PricePerSqFt'] = (df['SalePrice'] / df['SquareFeet']).round(2)
    df['PropertyAge'] = datetime.now().year - df['YearBuilt']

    return df

# Calculate market insights
def calculate_market_insights(df):
    """Calculate advanced market insights"""
    insights = {}

    if 'Year' in df.columns:
        yearly_avg = df.groupby('Year')['SalePrice'].mean()
        if len(yearly_avg) > 1:
            yoy_growth = ((yearly_avg.iloc[-1] - yearly_avg.iloc[-2]) / yearly_avg.iloc[-2] * 100)
            insights['yoy_growth'] = yoy_growth

    if 'PropertyType' in df.columns:
        type_avg = df.groupby('PropertyType')['SalePrice'].mean()
        insights['most_expensive_type'] = type_avg.idxmax()
        insights['most_expensive_price'] = type_avg.max()

    if 'PricePerSqFt' in df.columns and 'State' in df.columns:
        state_value = df.groupby('State')['PricePerSqFt'].mean()
        insights['best_value_state'] = state_value.idxmin()
        insights['best_value_price'] = state_value.min()

    if 'Year' in df.columns and len(df['Year'].unique()) > 1:
        recent_year = df['Year'].max()
        recent_avg = df[df['Year'] == recent_year]['SalePrice'].mean()
        older_avg = df[df['Year'] < recent_year]['SalePrice'].mean()
        insights['market_momentum'] = 'Up' if recent_avg > older_avg else 'Down'

    return insights

# Main app
def main():
    # Header
    col1, col2, col3 = st.columns([1, 2, 1])
    with col2:
        st.markdown('<h1 class="main-header">🏠 Real Estate USA Dashboard</h1>', unsafe_allow_html=True)

    # Sidebar
    st.sidebar.title("📊 Dashboard Controls")

    # Data source selection
    st.sidebar.subheader("🔌 Data Source")

    data_sources = ["Sample Data", "CSV Files"]
    if PYODBC_AVAILABLE:
        data_sources.insert(1, "SQL Server Database")

    data_source = st.sidebar.radio(
        "Choose your data source:",
        data_sources,
        help="Select where to load the real estate data from"
    )

    # Load data
    df = None

    if data_source == "Sample Data":
        with st.spinner("Loading sample data..."):
            df = generate_sample_data()
        st.sidebar.success("✅ Using sample data (1,500 properties)")

    elif data_source == "SQL Server Database":
        with st.spinner("Connecting to SQL Server..."):
            conn = get_database_connection()
            if conn:
                use_custom_query = st.sidebar.checkbox("Use custom SQL query", value=False)

                if use_custom_query:
                    custom_query = st.sidebar.text_area(
                        "Enter your SQL query:",
                        height=150,
                        placeholder="SELECT * FROM RealEstateSales WHERE ..."
                    )
                    if st.sidebar.button("Execute Query"):
                        df = load_data_from_db(conn, custom_query)
                else:
                    df = load_data_from_db(conn)

                if df is not None:
                    st.sidebar.success(f"✅ Connected! Loaded {len(df):,} records")
            else:
                st.sidebar.error("❌ Connection failed. Using sample data.")
                df = generate_sample_data()

    elif data_source == "CSV Files":
        with st.spinner("Loading CSV files..."):
            df = load_data_from_csv()
            if df is not None:
                st.sidebar.success(f"✅ Loaded {len(df):,} records from CSV")
            else:
                st.sidebar.warning("Using sample data instead")
                df = generate_sample_data()

    if df is None or df.empty:
        st.error("❌ No data available to display")
        return

    # Filters
    st.sidebar.markdown("---")
    st.sidebar.subheader("🔍 Filters")

    # Date filter
    if 'SaleDate' in df.columns:
        df_with_dates = df[df['SaleDate'].notna()]
        if not df_with_dates.empty:
            min_date = df_with_dates['SaleDate'].min().date()
            max_date = df_with_dates['SaleDate'].max().date()

            date_range = st.sidebar.date_input(
                "Date Range:",
                value=(min_date, max_date),
                min_value=min_date,
                max_value=max_date
            )

            if len(date_range) == 2:
                df = df[(df['SaleDate'].dt.date >= date_range[0]) &
                       (df['SaleDate'].dt.date <= date_range[1])]

    # State filter
    if 'State' in df.columns:
        states = ['All'] + sorted(df['State'].unique().tolist())
        selected_state = st.sidebar.selectbox("🗺️ Select State:", states)
        if selected_state != 'All':
            df = df[df['State'] == selected_state]

    # Property type filter
    if 'PropertyType' in df.columns:
        property_types = ['All'] + sorted(df['PropertyType'].unique().tolist())
        selected_property = st.sidebar.selectbox("🏘️ Property Type:", property_types)
        if selected_property != 'All':
            df = df[df['PropertyType'] == selected_property]

    # Price range filter
    if 'SalePrice' in df.columns:
        min_price = int(df['SalePrice'].min())
        max_price = int(df['SalePrice'].max())

        price_range = st.sidebar.slider(
            "💰 Price Range ($):",
            min_price,
            max_price,
            (min_price, max_price),
            format="$%d"
        )

        df = df[(df['SalePrice'] >= price_range[0]) &
               (df['SalePrice'] <= price_range[1])]

    # Bedrooms filter
    if 'Bedrooms' in df.columns:
        min_beds = int(df['Bedrooms'].min())
        max_beds = int(df['Bedrooms'].max())

        bedroom_range = st.sidebar.slider(
            "🛏️ Bedrooms:",
            min_beds,
            max_beds,
            (min_beds, max_beds)
        )

        df = df[(df['Bedrooms'] >= bedroom_range[0]) &
               (df['Bedrooms'] <= bedroom_range[1])]

    st.sidebar.markdown("---")
    st.sidebar.info(f"📊 **{len(df):,}** properties match your filters")

    if df.empty:
        st.warning("⚠️ No data matches your filter criteria. Please adjust your filters.")
        return

    # Calculate insights
    insights = calculate_market_insights(df)

    # Key Metrics
    st.subheader("📈 Key Performance Indicators")

    col1, col2, col3, col4, col5 = st.columns(5)

    with col1:
        total_sales = len(df)
        st.metric("Total Sales", f"{total_sales:,}")

    with col2:
        avg_price = df['SalePrice'].mean()
        delta_pct = insights.get('yoy_growth', 0)
        st.metric(
            "Average Price",
            f"${avg_price:,.0f}",
            f"{delta_pct:.1f}% YoY" if 'yoy_growth' in insights else None
        )

    with col3:
        median_price = df['SalePrice'].median()
        st.metric("Median Price", f"${median_price:,.0f}")

    with col4:
        if 'PricePerSqFt' in df.columns:
            avg_price_sqft = df['PricePerSqFt'].mean()
            st.metric("Avg $/Sq.Ft", f"${avg_price_sqft:.2f}")

    with col5:
        total_volume = df['SalePrice'].sum()
        st.metric("Total Volume", f"${total_volume/1e6:.1f}M")

    # Market Insights
    if insights:
        st.markdown("---")
        st.subheader("💡 Market Insights")

        col1, col2, col3 = st.columns(3)

        with col1:
            if 'most_expensive_type' in insights:
                st.markdown(f"""
                    <div class="info-box">
                        <h4>🏆 Most Valuable Type</h4>
                        <p><strong>{insights['most_expensive_type']}</strong></p>
                        <p>Avg: ${insights['most_expensive_price']:,.0f}</p>
                    </div>
                """, unsafe_allow_html=True)

        with col2:
            if 'best_value_state' in insights:
                st.markdown(f"""
                    <div class="info-box">
                        <h4>💎 Best Value Location</h4>
                        <p><strong>{insights['best_value_state']}</strong></p>
                        <p>${insights['best_value_price']:.2f}/sq.ft</p>
                    </div>
                """, unsafe_allow_html=True)

        with col3:
            if 'market_momentum' in insights:
                momentum_color = "#28a745" if insights['market_momentum'] == 'Up' else "#dc3545"
                st.markdown(f"""
                    <div class="info-box">
                        <h4>📊 Market Trend</h4>
                        <p><strong style="color: {momentum_color};">{insights['market_momentum']}ward</strong></p>
                        <p>Based on recent sales</p>
                    </div>
                """, unsafe_allow_html=True)

    # Tabs
    st.markdown("---")
    tab1, tab2, tab3 = st.tabs(["📊 Overview", "📈 Trends", "📋 Data Table"])

    with tab1:
        col1, col2 = st.columns(2)

        with col1:
            if 'State' in df.columns:
                sales_by_state = df.groupby('State').agg({
                    'SalePrice': ['count', 'sum', 'mean']
                }).reset_index()
                sales_by_state.columns = ['State', 'Count', 'Total', 'Average']
                sales_by_state = sales_by_state.sort_values('Total', ascending=False).head(10)

                fig1 = px.bar(
                    sales_by_state,
                    x='State',
                    y='Total',
                    title='Top 10 States by Sales Volume',
                    color='Total',
                    color_continuous_scale='Blues',
                    hover_data={'Count': True, 'Average': ':$,.0f'}
                )
                fig1.update_layout(height=450, showlegend=False)
                st.plotly_chart(fig1, use_container_width=True)

        with col2:
            if 'PropertyType' in df.columns:
                property_dist = df['PropertyType'].value_counts().reset_index()
                property_dist.columns = ['PropertyType', 'Count']

                fig2 = px.pie(
                    property_dist,
                    values='Count',
                    names='PropertyType',
                    title='Property Type Distribution',
                    hole=0.4
                )
                fig2.update_layout(height=450)
                st.plotly_chart(fig2, use_container_width=True)

        col1, col2 = st.columns(2)

        with col1:
            fig3 = px.histogram(
                df,
                x='SalePrice',
                nbins=50,
                title='Sale Price Distribution',
                color_discrete_sequence=['#1f77b4']
            )
            fig3.update_layout(height=400, showlegend=False)
            st.plotly_chart(fig3, use_container_width=True)

        with col2:
            if 'Bedrooms' in df.columns:
                bedroom_stats = df.groupby('Bedrooms').agg({
                    'SalePrice': ['count', 'mean']
                }).reset_index()
                bedroom_stats.columns = ['Bedrooms', 'Count', 'AvgPrice']

                fig4 = px.bar(
                    bedroom_stats,
                    x='Bedrooms',
                    y='AvgPrice',
                    title='Average Price by Bedrooms',
                    color='AvgPrice',
                    color_continuous_scale='Viridis'
                )
                fig4.update_layout(height=400, showlegend=False)
                st.plotly_chart(fig4, use_container_width=True)

    with tab2:
        st.subheader("📈 Time-Series Analysis")

        if 'Year' in df.columns and 'Month' in df.columns:
            monthly_data = df.groupby(['Year', 'Month']).agg({
                'SalePrice': ['count', 'mean']
            }).reset_index()
            monthly_data.columns = ['Year', 'Month', 'Count', 'AvgPrice']
            monthly_data['Period'] = pd.to_datetime(
                monthly_data['Year'].astype(str) + '-' + monthly_data['Month'].astype(str)
            )
            monthly_data = monthly_data.sort_values('Period')

            col1, col2 = st.columns(2)

            with col1:
                fig5 = px.line(
                    monthly_data,
                    x='Period',
                    y='AvgPrice',
                    title='Average Sale Price Trend',
                    markers=True
                )
                fig5.update_layout(height=400)
                st.plotly_chart(fig5, use_container_width=True)

            with col2:
                fig6 = px.bar(
                    monthly_data,
                    x='Period',
                    y='Count',
                    title='Sales Volume Trend',
                    color='Count',
                    color_continuous_scale='Greens'
                )
                fig6.update_layout(height=400, showlegend=False)
                st.plotly_chart(fig6, use_container_width=True)
        else:
            st.info("Time-series data not available")

    with tab3:
        st.subheader("📋 Property Listings")

        search_term = st.text_input("🔍 Search:", placeholder="Enter city, state, or property type...")

        display_df = df.copy()

        if search_term:
            mask = display_df.astype(str).apply(
                lambda row: row.str.contains(search_term, case=False, na=False).any(),
                axis=1
            )
            display_df = display_df[mask]

        all_columns = display_df.columns.tolist()
        default_columns = [col for col in ['SaleDate', 'State', 'City', 'PropertyType',
                                           'SalePrice', 'Bedrooms', 'Bathrooms', 'SquareFeet']
                          if col in all_columns]

        selected_columns = st.multiselect(
            "Select columns:",
            all_columns,
            default=default_columns
        )

        if selected_columns:
            display_df = display_df[selected_columns]

        col1, col2 = st.columns(2)

        with col1:
            sort_by = st.selectbox("Sort by:", display_df.columns.tolist())

        with col2:
            show_records = st.slider("Records:", 10, 100, 25)

        display_df = display_df.sort_values(by=sort_by, ascending=False)

        for col in display_df.columns:
            if pd.api.types.is_datetime64_any_dtype(display_df[col]):
                display_df[col] = display_df[col].dt.strftime('%Y-%m-%d')

        st.dataframe(display_df.head(show_records), use_container_width=True, height=500)

        csv = display_df.to_csv(index=False)
        st.download_button(
            label="📥 Download as CSV",
            data=csv,
            file_name=f"real_estate_data_{datetime.now().strftime('%Y%m%d')}.csv",
            mime="text/csv"
        )

    # Footer
    st.markdown("---")
    st.markdown("""
        <div style='text-align: center; color: gray;'>
        Real Estate Business Intelligence Dashboard | Built with Streamlit
        </div>
    """, unsafe_allow_html=True)

if __name__ == "__main__":
    main()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 83.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 340.3/340.3 kB 30.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 110.0 MB/s eta 0:00:00


2025-12-18 21:39:38.665 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-18 21:39:38.666 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-18 21:39:38.839 
  command:

    streamlit run /usr/local/lib/python3.12/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2025-12-18 21:39:38.840 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-18 21:39:38.841 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-18 21:39:38.845 No runtime found, using MemoryCacheStorageManager
2025-12-18 21:39:38.847 No runtime found, using MemoryCacheStorageManager
2025-12-18 21:39:38.850 No runtime found, using MemoryCacheStorageManager
2025-12-18 2

In [ ]:
from google.colab import drive
drive.mount('/content/drive')